In [8]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns

data = pd.read_csv("ORGANICS.csv", encoding = "cp1251", delimiter = ',')
print(data.shape) 
print(data.info())


(22223, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 13 columns):
ID                 22223 non-null int64
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
TargetAmt          22223 non-null int64
dtypes: float64(5), int64(3), object(5)
memory usage: 2.2+ MB
None


In [9]:
data = data.dropna()

In [11]:
x = data[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]
y = data.values[::,11]
y = y.astype('int')

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 4, random_state = 0)
tree.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best')

In [15]:
import sys
!{sys.executable} -m pip install pydot

In [20]:
conda install graphviz 

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [21]:
 pip install graphviz 

Note: you may need to restart the kernel to use updated packages.


In [22]:
 pip install pydot 

Note: you may need to restart the kernel to use updated packages.


In [23]:
from sklearn.tree import export_graphviz

import subprocess

export_graphviz(tree, out_file='Organics_tree.dot', 
               rounded = True, proportion = False, 
                precision = 2, filled = True)

import pydot 
(graph, ) = pydot.graph_from_dot_file('Organics_tree.dot')

graph.write_png('Organics_tree.png')


In [24]:
print(tree.score(X_train, y_train))
print(tree.score(X_test, y_test))

0.7971266869830214
0.7970749542961609


In [25]:
#a)	Обучите решающее дерево с параметрами criterion = entropy и max_depth =5.  Какую точность дает данное дерево на контроле? 
#Визуализируйте разделяющую поверхность.
newTree = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')
newTree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [26]:
from sklearn.tree import export_graphviz

import subprocess

export_graphviz(newTree, out_file = 'Organics_newTree.dot', 
               rounded = True, proportion = False, 
                precision = 2, filled = True)

import pydot 
(graph, ) = pydot.graph_from_dot_file('Organics_newTree.dot')

graph.write_png('Organics_newTree.png')


In [27]:
print(newTree.score(X_train, y_train))
print(newTree.score(X_test, y_test))

0.8003482803656944
0.7964655697745278


In [37]:
#b)	Выполните предыдущий пункт для разных значений глубины дерева и минимального числа объектов в листе 
#(параметры max_depth и min_samples_leaf). Результаты запишите в виде таблицы.
tree_one = DecisionTreeClassifier(max_depth = 2, min_samples_leaf = 4)
tree_one.fit(X_train, y_train)
one = tree_one.score(X_train, y_train)

tree_two = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 2)
tree_two.fit(X_train, y_train)
two = tree_two.score(X_train, y_train)

tree_three = DecisionTreeClassifier(max_depth = 1, min_samples_leaf = 5)
tree_three.fit(X_train, y_train)
three = tree_three.score(X_train, y_train)

tree_four = DecisionTreeClassifier(max_depth = 5, min_samples_leaf = 1)
tree_four.fit(X_train, y_train)
four = tree_four.score(X_train, y_train)

tree_five = DecisionTreeClassifier(max_depth = 6, min_samples_leaf = 6)
tree_five.fit(X_train, y_train)
five = tree_five.score(X_train, y_train)

table = {
'max_depth': [2, 4, 1, 5, 6],
'min_samples_leaf': [4, 2, 5, 1, 6],
'values': [one, two, three, four, five]
}
print_table = pd.DataFrame(data = table, index = ['first', 'second', 'third', 'fourth', 'fiveth'])

print_table

,max_depth,min_samples_leaf,values
first,2,4,0.790335
second,4,2,0.797127
third,1,5,0.760296
fourth,5,1,0.800348
fiveth,6,6,0.803396


In [57]:
#c)	Реализуйте бинарное кодирование категориальных признаков. Если у признака N возможных значений, то ему в соответствие 
#ставится N бинарных признаков, каждый из которых соответствует одному значению исходного признака. 
#На каждом объекте единице должен равняться строго один из них. Примените такое кодирование к столбцам 
#c категориальными данными, добавьте результат к признакам. 
data_le = data[['DemAffl',  'DemAge']]

def conj(data, column_one, column_two):
    data[column_one + '+' + column_two] = data[column_one] + data[column_two]
    return (data)

from sklearn.preprocessing import LabelEncoder

le_one = LabelEncoder()
le_one.fit(data_le.DemAffl)

le_second = LabelEncoder()
le_second.fit(data_le.DemAge)

data_le['DemAffl_le'] = le_one.transform(data_le.DemAffl)
data_le['DemAge_le'] = le_second.transform(data_le.DemAge)
conj(data_le, 'DemAffl', 'DemAffl_le')
conj(data_le, 'DemAge', 'DemAge_le')
data_le

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,DemAffl,DemAge,DemAffl_le,DemAge_le,DemAffl+DemAffl_le,DemAge+DemAge_le
0,10.0,76.0,10,58,20.0,134.0
1,4.0,49.0,4,31,8.0,80.0
2,5.0,70.0,5,52,10.0,122.0
3,10.0,65.0,10,47,20.0,112.0
4,11.0,68.0,11,50,22.0,118.0
...,...,...,...,...,...,...
22216,13.0,49.0,13,31,26.0,80.0
22218,13.0,65.0,13,47,26.0,112.0
22219,15.0,73.0,15,55,30.0,128.0
22220,9.0,70.0,9,52,18.0,122.0


In [58]:
#2)	 Построить дерево решений для датасетов по вариантам, оформить блокноты с описанием задачи, признаков:        
#а) titanic.csv  (Описание https://www.kaggle.com/c/titanic/data)
titanic = pd.read_csv("titanic.csv", encoding = "cp1251", delimiter = ',')
print(titanic.shape) 
print(titanic.info())

(887, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
Survived                   887 non-null int64
Pclass                     887 non-null int64
Name                       887 non-null object
Sex                        887 non-null object
Age                        887 non-null float64
Siblings/Spouses Aboard    887 non-null int64
Parents/Children Aboard    887 non-null int64
Fare                       887 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.6+ KB
None


In [59]:
titanic

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


In [60]:
titanic = titanic.dropna()

In [61]:
titanic

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
882,0,2,Rev. Juozas Montvila,male,27.0,0,0,13.0000
883,1,1,Miss. Margaret Edith Graham,female,19.0,0,0,30.0000
884,0,3,Miss. Catherine Helen Johnston,female,7.0,1,2,23.4500
885,1,1,Mr. Karl Howell Behr,male,26.0,0,0,30.0000


In [62]:
x = titanic[['Pclass', 'Age', 'Fare']]
y = titanic.values[::,0]
y = y.astype('int')

In [63]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

tree = DecisionTreeClassifier(max_depth = 4, min_samples_leaf = 2)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [64]:
export_graphviz(tree, out_file='titanic.dot', 
               rounded = True, proportion = False, 
                precision = 2, filled = True)

(graph, ) = pydot.graph_from_dot_file('titanic.dot')

graph.write_png('titanic.png')


In [65]:
print(tree.score(X_train, y_train))
print(tree.score(X_test, y_test))

0.7016129032258065
0.6292134831460674
